# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csvpath = os.path.join("../output_data/city_data.csv")
city_df = pd.read_csv(csvpath)
del city_df['Unnamed: 0']
city_df["date"] = pd.to_datetime(city_df['date'], unit='s')
city_df.head(100)

city country                date    lat     lon   temp  hum  \
0   gornopravdinsk      RU 2020-04-11 18:25:00  60.05   69.90  35.87   87   
1           tiznit      MA 2020-04-11 18:20:01  29.58   -9.50  62.62   60   
2         thompson      CA 2020-04-11 18:25:01  55.74  -97.86  26.60   68   
3         hermanus      ZA 2020-04-11 18:25:01 -34.42   19.23  67.75   76   
4    west wendover      US 2020-04-11 18:25:01  40.74 -114.07  66.20   24   
..             ...     ...                 ...    ...     ...    ...  ...   
95       coihaique      CL 2020-04-11 18:25:08 -45.58  -72.07  50.47   48   
96            hilo      US 2020-04-11 18:25:08  19.73 -155.09  69.31   73   
97        nokaneng      BW 2020-04-11 18:25:08 -19.67   22.27  76.06   42   
98         chambas      CU 2020-04-11 18:25:08  22.19  -78.91  89.60   46   
99           alofi      NU 2020-04-11 18:25:08 -19.06 -169.92  66.20  100   

    clouds   wind  
0        0   6.51  
1      100   7.76  
2       90   5.82  
3      100   3.00  
4        1  16.11  
..     ...    ...  
95      40   8.05  
96       1   3.36  
97      83   4.03  
98       0  10.29  
99     100   3.36  

[100 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["lat", "lon"]]
hum = city_df["hum"].astype(float)

kasia_center = (20, 10)


fig_city = gmaps.figure(center=kasia_center, zoom_level=1.5)
figure_layout = {
    'width': '200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px'
}
gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=hum, 
                                 dissipating=False, max_intensity=150,
                                 point_radius=2)
fig_city.add_layer(heat_layer)
fig_city


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
# A max temperature lower than 90 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

ideal_vacay = city_df.loc[(city_df["temp"] >=70)& (city_df["temp"] <=90) & (city_df["wind"] <=10) & (city_df["clouds"] ==0)]
ideal_vacay = ideal_vacay.dropna(how='any')
ideal_vacay_df = ideal_vacay.reset_index(drop=True)
ideal_vacay_df[f"Hotel Name"] = ""
ideal_vacay_df


city country                date    lat     lon   temp  hum  \
0  machilipatnam      IN 2020-04-11 18:25:25  16.17   81.13  80.28   79   
1          asyut      EG 2020-04-11 18:25:26  27.18   31.18  75.20   36   
2        dolores      AR 2020-04-11 18:24:06 -36.31  -57.68  73.72   42   
3      roebourne      AU 2020-04-11 18:25:33 -20.78  117.13  82.06   50   
4         palasa      IN 2020-04-11 18:25:36  18.77   84.42  80.89   73   
5   port hedland      AU 2020-04-11 18:25:38 -20.32  118.57  73.40   83   

   clouds  wind Hotel Name  
0       0  8.39             
1       0  9.17             
2       0  6.71             
3       0  9.89             
4       0  8.81             
5       0  5.82

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df = ideal_vacay_df 

params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lon"]
    
    location = row["city"]
    print(f"Retrieving Results for Index {index}: {location}.")

    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"  
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    

    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    
    except (KeyError, IndexError):        
        print("No hotel within 5000 meters")     
    
    print("______________________________________________________")


Retrieving Results for Index 0: machilipatnam.
Closest hotel is Teja Srinivas Residency.
______________________________________________________
Retrieving Results for Index 1: asyut.
Closest hotel is Florence Hotel Assiut.
______________________________________________________
Retrieving Results for Index 2: dolores.
Closest hotel is HOTEL TERMAL HOWARD JOHNSON DOLORES.
______________________________________________________
Retrieving Results for Index 3: roebourne.
Closest hotel is Latitude20 Roebourne Village.
______________________________________________________
Retrieving Results for Index 4: palasa.
Closest hotel is Hotels at kssibugga.
______________________________________________________
Retrieving Results for Index 5: port hedland.
Closest hotel is ibis Styles Port Hedland.
______________________________________________________


In [13]:
hotel_df.columns=["City Name","Country","Date","Lat","Lon","Temp","Hum","Clouds","Wind","Hotel Name"]
hotel_df

City Name Country                Date    Lat     Lon   Temp  Hum  \
0  machilipatnam      IN 2020-04-11 18:25:25  16.17   81.13  80.28   79   
1          asyut      EG 2020-04-11 18:25:26  27.18   31.18  75.20   36   
2        dolores      AR 2020-04-11 18:24:06 -36.31  -57.68  73.72   42   
3      roebourne      AU 2020-04-11 18:25:33 -20.78  117.13  82.06   50   
4         palasa      IN 2020-04-11 18:25:36  18.77   84.42  80.89   73   
5   port hedland      AU 2020-04-11 18:25:38 -20.32  118.57  73.40   83   

   Clouds  Wind                           Hotel Name  
0       0  8.39              Teja Srinivas Residency  
1       0  9.17                Florence Hotel Assiut  
2       0  6.71  HOTEL TERMAL HOWARD JOHNSON DOLORES  
3       0  9.89         Latitude20 Roebourne Village  
4       0  8.81                  Hotels at kssibugga  
5       0  5.82             ibis Styles Port Hedland

In [14]:
info_box_template = """
# <dl>
# <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
# <dt>City Name</dt><dd>{City Name}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat","Lon"]]


In [15]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig_city.add_layer(marker_layer)
fig_city

Figure(layout=FigureLayout(height='420px'))